Author: Deepu Variyangattil

In [31]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import queue
import re
import csv

In [40]:
options = webdriver.ChromeOptions()
options.add_argument("headless")

driver = webdriver.Chrome(executable_path='F:\Stevens_Masters\Spring 19\Web Mining\Week 3\chromedriver_win32\chromedriver.exe')



In [106]:
driver.get("https://www.delish.com/")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [109]:
links=driver.find_elements_by_xpath("//*[@class='nav-item']/a")


In [110]:
q=queue.Queue()
for link in links:
    a=link.get_attribute('href')
    
    q.put(a)
    q.put("https://www.delish.com/cooking/")
    
    

In [111]:
receipe_Queue=queue.Queue()
while q.qsize()>0:
    url=q.get()
    
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    receipeLinks=driver.find_elements_by_xpath("//*[@class='full-item-content']/a")
    
    for receipe in receipeLinks:
        li=receipe.get_attribute('href')
        
        if re.match('^(.)+recipe\/$',li) is not None:
            receipe_Queue.put(li)
            
            
print("Initial Queue empty")        

Initial Queue empty


In [114]:
mydata=[["RECIPE","INGREDIENTS","URL"]]
myFile = open("recipelist.csv", 'w',newline='')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(mydata)

In [115]:
url_List=[]
while receipe_Queue.qsize()>0:
    url=receipe_Queue.get()
    if url not in url_List:
        url_List.append(url)
        driver.get(url)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        recipe_Name=driver.find_element_by_xpath("//*[@class='content-hed recipe-hed']").text

        ing_List=driver.find_elements_by_xpath("//*[@class='ingredient-description']")
    
        s=""
        for ingre in ing_List:
        
        
            p=ingre.text
            p=p.replace(',',"")
            if s=="":
                s=p
            else:
                s=s+","+p
        
    

        mydata=[[recipe_Name,s,url]]
        myFile = open("recipelist.csv", 'a',newline='',encoding="utf-8")
        with myFile:
            writer = csv.writer(myFile)
            writer.writerows(mydata)
print("Queue finished")
myFile.close()

Queue finished


In [116]:
driver.close()

In [117]:
print(len(url_List))

48
